In [1]:
!python --version

Python 3.9.17


In [2]:
import os
import pickle
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.feature_extraction import DictVectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import mlflow
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient

In [3]:
from sklearn.pipeline import make_pipeline

In [4]:
from sklearn.linear_model import LinearRegression

In [5]:
from google.cloud import storage
client = storage.Client()

In [29]:
TRACKING_SERVER_HOST = "34.16.191.116"
TRACKING_SERVER_PORT = "5000"
DATA_PATH = "~/data/day.csv"
EXPERIMENT_NAME = "bike-sharing-regression"

In [12]:
TRACKING_URI=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"

In [15]:
def load_data(filename):
    df = pd.read_csv(filename, sep=',')
    
    # drop columns which are not required for training
    df = df.drop(['instant', 'dteday', 'yr', 'casual', 'registered'], axis=1)
    
    return df

In [8]:
def split_data(df):
    df_train, df_val = train_test_split(df, test_size=0.2, random_state=42,
                                        stratify=df[['season', 'weekday']])
    return df_train, df_val

In [9]:
def prepare_dictionaries(df: pd.DataFrame):
    features = ['season', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit',
                        'temp', 'atemp', 'hum', 'windspeed']
    return df[features].to_dict(orient='records')

In [10]:
def train_model_rf_search(dict_train, dict_val, y_train, y_val):
    mlflow.sklearn.autolog()

    def objective(params):
        with mlflow.start_run():
            mlflow.set_tag("model", "rf")
            mlflow.log_param("train_data",DATA_PATH)
            
            pipeline = make_pipeline(
                DictVectorizer(),
                RandomForestRegressor(**params, n_jobs=-1)
            )
            
            pipeline.fit(dict_train, y_train)

            y_pred = pipeline.predict(dict_val)
            rmse = mean_squared_error(y_val, y_pred, squared=False)
            r2score = r2_score(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2_score", r2score)
                
            mlflow.sklearn.log_model(pipeline, artifact_path="models")

        return {'loss': rmse, 'status': STATUS_OK}


    search_space = {
        'n_estimators' : scope.int(hp.uniform('n_estimators',10,150)),
        'max_depth' : scope.int(hp.uniform('max_depth',1,40)),
        'min_samples_leaf' : scope.int(hp.uniform('min_samples_leaf',1,10)),
        'min_samples_split' : scope.int(hp.uniform('min_samples_split',2,10)),
        'random_state' : 42
    }
    
    rstate = np.random.default_rng(42)  # for reproducible results
    best_result =  fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=200,
        trials=Trials(),
        rstate=rstate
    )
    return

In [7]:
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='gs://mlflow-assignment-mj/bike-sharing-prediction/3', creation_time=1691221899736, experiment_id='3', last_update_time=1691221899736, lifecycle_stage='active', name='bike-sharing-regression', tags={}>

In [12]:
df = load_data(DATA_PATH)

In [13]:
df_train, df_val = split_data(df)

In [14]:
dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [15]:
target = 'cnt'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
train_model_rf_search(dict_train, dict_val, y_train, y_val)

  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]

2023/08/05 08:22:30 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/05 08:22:33 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/05 08:22:36 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  0%|          | 1/200 [00:09<31:00,  9.35s/trial, best loss: 1192.221603892011]

2023/08/05 08:22:39 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/05 08:22:42 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/05 08:22:45 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  1%|          | 2/200 [00:18<29:54,  9.06s/trial, best loss: 1192.221603892011]

...

 99%|█████████▉| 198/200 [30:26<00:18,  9.30s/trial, best loss: 1187.4476830955357]

2023/08/05 08:52:56 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/05 08:52:59 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/05 08:53:02 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



100%|█████████▉| 199/200 [30:35<00:09,  9.22s/trial, best loss: 1187.4476830955357]

2023/08/05 08:53:05 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/08/05 08:53:08 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/08/05 08:53:11 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



100%|██████████| 200/200 [30:44<00:00,  9.22s/trial, best loss: 1187.4476830955357]


In [8]:
client = MlflowClient(tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [11]:
def register_best_model(tracking_uri, experiment_name, model_registry_name):
    
    client = MlflowClient(tracking_uri=tracking_uri)
    
    experiment = client.get_experiment_by_name(experiment_name)
    best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.rmse ASC"]
    )[0]
    
    # register the best model
    run_id = best_run.info.run_id
    model_uri = f"runs:/{run_id}/model"
    model_rmse = best_run.data.metrics['rmse']
    model_details = mlflow.register_model(model_uri=model_uri, name=model_registry_name)

    date = datetime.today().date()
    
    # transition of our best model in "Production"
    client.transition_model_version_stage(
        name=model_details.name,
        version=model_details.version,
        stage="Production",
        archive_existing_versions=True
    )
    
    client.update_model_version(
        name=model_details.name,
        version=model_details.version,
        description=f"The model version {model_details.version} was transitioned to Production on {date}"
    )
    
    client.update_registered_model(
        name=model_details.name,
        description=f"Current model version in production: {model_details.version}, rmse: {model_rmse}"
    )

In [13]:
model_name = "random-forest-regressor"

In [14]:
register_best_model(TRACKING_URI, EXPERIMENT_NAME, model_name)

Registered model 'random-forest-regressor' already exists. Creating a new version of this model...
2023/08/07 17:52:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-regressor, version 2
Created version '2' of model 'random-forest-regressor'.


In [9]:
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)[0]
best_run

<Run: data=<RunData: metrics={'r2_score': 0.632749344020741,
 'rmse': 1187.4476830955357,
 'training_mean_absolute_error': 839.3195837883158,
 'training_mean_squared_error': 984957.1543581459,
 'training_r2_score': 0.7355399970094947,
 'training_root_mean_squared_error': 992.4500765066956,
 'training_score': 0.7355399970094947}, params={'dictvectorizer': 'DictVectorizer()',
 'dictvectorizer__dtype': "<class 'numpy.float64'>",
 'dictvectorizer__separator': '=',
 'dictvectorizer__sort': 'True',
 'dictvectorizer__sparse': 'True',
 'memory': 'None',
 'randomforestregressor': 'RandomForestRegressor(max_depth=9, '
                          'min_samples_leaf=9, min_samples_split=7,\n'
                          '                      n_estimators=72, n_jobs=-1, '
                          'random_state=42)',
 'randomforestregressor__bootstrap': 'True',
 'randomforestregressor__ccp_alpha': '0.0',
 'randomforestregressor__criterion': 'squared_error',
 'randomforestregressor__max_depth': '9',
 'r

In [28]:
run_id = best_run.info.run_id
model_uri=f"runs:/{run_id}/models"


In [23]:
print(model_uri)

runs:/79934c79a98f4932aade316cce6e61a0/models


In [24]:
mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'random-forest-regressor'.
2023/08/05 13:18:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-regressor, version 1
Created version '1' of model 'random-forest-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1691241529170, current_stage='None', description='', last_updated_timestamp=1691241529170, name='random-forest-regressor', run_id='79934c79a98f4932aade316cce6e61a0', run_link='', source='gs://mlflow-assignment-mj/bike-sharing-prediction/3/79934c79a98f4932aade316cce6e61a0/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [29]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1691241529170, current_stage='Production', description='', last_updated_timestamp=1691241730726, name='random-forest-regressor', run_id='79934c79a98f4932aade316cce6e61a0', run_link='', source='gs://mlflow-assignment-mj/bike-sharing-prediction/3/79934c79a98f4932aade316cce6e61a0/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [30]:
latest_versions = client.get_latest_versions(name=model_name)

In [37]:
prod_model_uri = f"models:/{model_name}/latest"

In [38]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [40]:
# testing prediction
bike_data = {
    'season': 1,
    'mnth': 1,
    'holiday': 0,
    'weekday': 6, 
    'workingday': 0, 
    'weathersit': 2,
    'temp': 0.344167, 
    'atemp': 0.363625,
    'hum': 0.805833,
    'windspeed': 0.160446
}

In [41]:
loaded_model.predict(bike_data)

array([2475.51869788])